In [1]:
import pandas as pd
import numpy as np
import json
import os

Foram removidas alguns valores aleatórios do labelled.json por conterem erros ao aceder-lhes - certas linhas tinham mais valores do que deviam

In [2]:
import parse

In [3]:
annotations_path = os.path.join('../annotations')
json_data_path = os.path.join('results', 'labelled.json')
data_path = os.path.join('results', 'labelled.csv')

In [4]:
# parse.parse_annotation(annotations_path, json_data_path)

In [5]:
def get_data_from_json():
    with open(json_data_path, 'r') as input:
        data = json.load(input)

    columns = ['piece', 'name', 'midi', 'measures', 'duration', 'arousal', 'valence']
    df = pd.DataFrame(columns=columns)

    for piece in data.keys():
        name = data[piece]['name']
        midi = data[piece]['midi']
        measures = data[piece]['measures']
        duration = data[piece]['duration']
        arousal = np.array(data[piece]['arousal'], dtype=float).mean()
        valence = np.array(data[piece]['valence'], dtype=float).mean()
        series = pd.Series([piece, name, midi, measures, duration, arousal, valence], index=df.columns)
        df = pd.concat([df, series.to_frame().T], ignore_index=True)
    df.duration = df.duration.astype('int64')
    df.measures = df.measures.astype('int64')
    df["arousal"] = pd.to_numeric(df["arousal"], downcast="float")
    df["valence"] = pd.to_numeric(df["valence"], downcast="float")
    df.to_csv(data_path, index=False)
    return df


In [6]:
def get_data():
    if os.path.exists(data_path):
        return pd.read_csv(data_path)
    else:
        return get_data_from_json()

In [10]:
data = get_data()

In [11]:
data

,piece,name,midi,measures,duration,arousal,valence
0,piece_0,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,32,63,-0.032802,-0.203875
1,piece_1,Continue?,Final Fantasy_PS1_Final Fantasy VII_Continue.mid,25,74,-0.207000,0.151760
2,piece_2,Oppressed People,Final Fantasy_PS1_Final Fantasy VII_Oppressed ...,40,70,0.058008,-0.138137
3,piece_3,Deku Palace,The Legend of Zelda_N64_The Legend of Zelda Ma...,60,131,0.189081,0.446306
4,piece_4,Holding My Thoughts in My Heart,Final Fantasy_PS1_Final Fantasy VII_Holding My...,34,116,-0.211324,0.240137
...,...,...,...,...,...,...,...
195,piece_195,Maguss Theme,Chrono_SNES_Chrono Trigger_Maguss Theme.mid,71,149,0.147208,-0.019774
196,piece_196,Zeppelin,Indiana Jones_PC_Indiana Jones and the Last Cr...,35,69,0.097327,0.300337
197,piece_197,JENOVA,Final Fantasy_PS1_Final Fantasy VII_JENOVA.mid,100,133,0.376296,0.012214
198,piece_198,Isle O Hags Cauldron Keep,Banjo-Kazooie_N64_Banjo-Tooie_Isle O Hags Caul...,62,123,-0.012431,-0.094902


## Append emotion labels

![Emotions - Russell circumplex](images/emotions_circumplex.png)

In [12]:
happy_indexes = (data.arousal >= 0) & (data.valence >= 0)
sad_indexes = (data.arousal <= 0) & (data.valence <= 0)
relaxed_indexes = (data.valence > 0) & (data.arousal < 0)
distress_indexes = (data.arousal > 0) & (data.valence < 0)

In [13]:
data['emotion'] = 'something'
data.loc[happy_indexes, 'emotion'] = 'joy'
data.loc[sad_indexes, 'emotion'] = 'sad'
data.loc[relaxed_indexes, 'emotion'] = 'relaxed'
data.loc[distress_indexes, 'emotion'] = 'distress'

In [ ]:
# mudar labels para 1, 2, 3, 4

In [14]:
data.emotion.unique()

array(['sad', 'relaxed', 'distress', 'joy'], dtype=object)

In [15]:
data.to_csv('../dataset/dataset.csv', index=False)